In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv


# Install Imports

In [2]:
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import scipy as sp
import seaborn as sns
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import polars as pl
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
np.random.seed(42)

# Load Dataset

In [4]:
train = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
submission = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")

In [5]:
train.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [6]:
test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


# Feature Engineering

In [7]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

In [8]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html = re.compile(r'<.*?>')
    return html.sub(r'',x) # html -> ''

def dataPreprocessing(x):
    
    x = x.lower()
    # remove html
    x = removeHTML(x)
    
    #remove string with starting @
    x = re.sub("@\w+",'',x)
    
    # remove number
    x = re.sub("'\d+",'',x)
    x = re.sub("\d+",'',x)
    
    # remove url
    x = re.sub("http\w+",'',x)
    
    # Replace consecutive empty spaces with a single empty spaces
    x = re.sub(r'\s+', " ", x)
    # Replace consecutive commas and periods with a single comma and period
    x = expandContractions(x)
    x = re.sub(r'\.+', ".",x)
    x = re.sub(r'\,+', ",",x)
    x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
    # Remove empty character at the beginning and end
    x = x.strip()
    return x

In [9]:
train = pl.from_pandas(train)

train = train.with_columns(
         pl.col('full_text').map_elements(lambda x: dataPreprocessing(x))
)
train = train.to_pandas()

/tmp/ipykernel_18/4137260655.py:3: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  train = train.with_columns(


In [10]:
test = pl.from_pandas(test)

test = test.with_columns(
         pl.col('full_text').map_elements(lambda x: dataPreprocessing(x))
)
test = test.to_pandas()

/tmp/ipykernel_18/1235424326.py:3: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  test = test.with_columns(


In [11]:
train["full_text"][0]

'many people have car where they live. the thing they do not know is that when you use a car alot of thing can happen like you can get in accidet or the smoke that the car has is bad to breath on if someone is walk but in vauban,germany they dont have that proble because percent of vaubans families do not own cars,and percent sold a car to move there. street parkig ,driveways and home garages are forbidden on the outskirts of freiburd that near the french and swiss borders. you probaly will not see a car in vaubans streets because they are completely "car free" but if some that lives in vauban that owns a car ownership is allowed,but there are only two places that you can park a large garages at the edge of the development,where a car owner buys a space but it not cheap to buy one they sell the space for you car for , along with a home. the vauban people completed this in ,they said that this an example of a growing trend in europe,the untile states and some where else are suburban lif

## Feature Engineering

In [12]:
class TextFeatureExtractor:
    def __init__(self):
        pass
    
    def word_count(self, text):
        return len(text.split())
    
    def sentence_count(self, text):
        return len(text.split('.'))
    
    def essay_length(self, text):
        return len(text)
    
    def extract_features(self, text):
        features = {
            'word_count': self.word_count(text),
            'sentence_count': self.sentence_count(text),
            'essay_length': self.essay_length(text)
        }
        return features

In [13]:
def apply_text_features(df, text_column):
    extractor = TextFeatureExtractor()
    for feature in ['word_count', 'sentence_count', 'essay_length']:
        df[feature] = df[text_column].apply(lambda x: getattr(extractor, feature)(x))

apply_text_features(train, 'full_text')
apply_text_features(test, 'full_text')

In [14]:
train

,essay_id,full_text,score,word_count,sentence_count,essay_length
0,000d118,many people have car where they live. the thin...,3,494,14,2642
1,000fe60,i am a scientist at nasa that is discussing th...,3,335,18,1665
2,001ab80,people always wish they had the same technolog...,4,550,25,3061
3,001bdc0,"we all heard about venus, the planet without a...",4,445,21,2665
4,002ba53,"dear, state senator this is a letter to argue ...",3,372,16,2168
...,...,...,...,...,...,...
17302,ffd378d,"the story "" the challenge of exploing venus "" ...",2,157,10,854
17303,ffddf1f,technology has changed a lot of ways that we l...,4,574,29,3313
17304,fff016d,if you do not like sitting around all day than...,2,220,16,1125
17305,fffb49b,"in ""the challenge of exporing venus,"" the auth...",1,227,12,1409


In [15]:
test

,essay_id,full_text,word_count,sentence_count,essay_length
0,000d118,many people have car where they live. the thin...,494,14,2642
1,000fe60,i am a scientist at nasa that is discussing th...,335,18,1665
2,001ab80,people always wish they had the same technolog...,550,25,3061


# Preparing Data

In [16]:
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix

# Exclude 'essay_id' from features
features_scale = ['word_count', 'sentence_count', 'essay_length']
scaler = StandardScaler()

# Scaling
train[features_scale] = scaler.fit_transform(train[features_scale])
test[features_scale] = scaler.transform(test[features_scale])

In [17]:
train.head()

,essay_id,full_text,score,word_count,sentence_count,essay_length
0,000d118,many people have car where they live. the thin...,3,0.830949,-0.684363,0.703004
1,000fe60,i am a scientist at nasa that is discussing th...,3,-0.224303,-0.205395,-0.432615
2,001ab80,people always wish they had the same technolog...,4,1.202610,0.632798,1.190030
3,001bdc0,"we all heard about venus, the planet without a...",4,0.505746,0.153830,0.729738
4,002ba53,"dear, state senator this is a letter to argue ...",3,0.021259,-0.444879,0.152049


In [18]:
test.head()

,essay_id,full_text,word_count,sentence_count,essay_length
0,000d118,many people have car where they live. the thin...,0.830949,-0.684363,0.703004
1,000fe60,i am a scientist at nasa that is discussing th...,-0.224303,-0.205395,-0.432615
2,001ab80,people always wish they had the same technolog...,1.202610,0.632798,1.190030


In [19]:
# Vectorize text data
tfid_vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = tfid_vectorizer.fit_transform(train['full_text'])

# Convert scaled features into sparse format
features_train = csr_matrix(train[features_scale])

# Combine TF-IDF features with additional features
X_train_combined = hstack([X_train_tfidf, features_train])

In [20]:
# Vectorize text data
X_test_tfidf = tfid_vectorizer.transform(test['full_text'])  

# Convert scaled features into sparse format
features_test = csr_matrix(test[features_scale])

# Combine TF-IDF features with additional features
X_test_combined = hstack([X_test_tfidf, features_test])

## Split data

In [21]:
y = train['score']

In [22]:
X_train,X_val,y_train,y_val = train_test_split(X_train_combined,y,test_size = 0.2)

## Model Selection + Training

In [23]:
classifier = SVC()

In [24]:
classifier.fit(X_train,y_train)

SVC()

In [25]:
val_pred = classifier.predict(X_val)

In [26]:
# Calculate test metrics
val_accuracy = accuracy_score(y_val, val_pred)
val_precision = precision_score(y_val, val_pred, average='macro')  
val_recall = recall_score(y_val, val_pred, average='macro') 
val_f1 = f1_score(y_val, val_pred, average='macro')  

# Print test metrics
print("Test Accuracy:", val_accuracy)
print("Test Precision:", val_precision)
print("Test Recall:", val_recall)
print("Test F1 Score:", val_f1)

Test Accuracy: 0.6062969381860196
Test Precision: 0.6112535080144684
Test Recall: 0.416456916595465
Test F1 Score: 0.42172544861889866


In [27]:
# Cohen's Kappa Score
val_kappa = cohen_kappa_score(y_val, val_pred)
print("Cohen's Kappa Score:", val_kappa)

Cohen's Kappa Score: 0.4467467939154066


## Submission

In [28]:
submission['score'] = classifier.predict(X_test_combined)

In [29]:
submission.to_csv("submission.csv",index = False)